# Create the preprocessed dataset

Do minor cleaning and extract the sheets from the original excel file.  
Save the files in 02_processed and also save an additional file that joins all the sheets together.

In [4]:
import numpy as np
import os
import pandas as pd
from pathlib import Path
import yaml

# Get the current project path (where you open the notebook)
# and go up two levels to get the project path
current_dir = Path.cwd()
proj_path = current_dir.parent.parent

# make the code in src available to import in this notebook
import sys
sys.path.append(os.path.join(proj_path, 'src'))

from utils import create_folder

# Catalog contains all the paths related to datasets
with open(os.path.join(proj_path, 'conf/catalog.yml'), "r") as f:
    catalog = yaml.safe_load(f)['breakfast']
    
# Params contains all of the dataset creation parameters and model parameters
with open(os.path.join(proj_path, 'conf/params.yml'), "r") as f:
    params = yaml.safe_load(f)

In [5]:
%%time

print('This may take a few minutes...')

main_fname = os.path.join(proj_path, catalog['base_dir'], catalog['xlsx_fname'])

# Skip the first row and select columns 0 to 11 inclusively or else 
# it will include extra empty columns due to the way the excel file is made
print('Reading transactions sheet')
transactions = pd.read_excel(main_fname,
                             skiprows=1,
                             usecols=np.arange(12),
                             sheet_name=catalog['sheet_names']['transactions'])
print('Reading products sheet')
products_lookup = pd.read_excel(main_fname,
                                skiprows=1,
                                usecols=np.arange(6),
                                sheet_name=catalog['sheet_names']['products'])

# For two stores 17627 and 4503, there are two values for the 
# column SEG_VALUE_NAME. Both 'MAINSTREAM' and 'UPSCALE'. By removing
# the first two instances of those records, we keep the value as UPSCALE
print('Reading store lookup sheet')
store_lookup = pd.read_excel(main_fname,
                             skiprows=1,
                             usecols=np.arange(9),
                             sheet_name=catalog['sheet_names']['store'])
store_lookup.drop(index=[22,39], inplace=True)
store_lookup.reset_index(drop=True, inplace=True)

print('Reading glossary sheet')
glossary = pd.read_excel(main_fname,
                         skiprows=3,
                         usecols=np.arange(3),
                         sheet_name=catalog['sheet_names']['glossary'],
                         names=['VARIABLE NAME', 'TABLE', 'DESCRIPTION'])

print('Writing files ...')
# Write those sheets as separate CSV files
create_folder(os.path.join(proj_path, catalog['output_dir']['dir']))
transactions.to_csv(os.path.join(proj_path, catalog['output_dir']['dir'], catalog['output_dir']['transactions']))
products_lookup.to_csv(os.path.join(proj_path, catalog['output_dir']['dir'],catalog['output_dir']['products']))
store_lookup.to_csv(os.path.join(proj_path, catalog['output_dir']['dir'],catalog['output_dir']['store']))
glossary.to_csv(os.path.join(proj_path, catalog['output_dir']['dir'],catalog['output_dir']['glossary']))


print(f"Wrote files to {os.path.join(proj_path, catalog['output_dir']['dir'])}")

This may take a few minutes...
Reading transactions sheet
Reading products sheet
Reading store lookup sheet
Reading glossary sheet
Writing files ...
Wrote files to C:\Users\feras\Desktop\ForecastingRetailSales\ForecastingRetailSales\data/02_processed/
Wall time: 8min 21s


In [3]:
%%time
merged_data = transactions.merge(products_lookup, on='UPC',how='left').merge(store_lookup,left_on='STORE_NUM',right_on='STORE_ID',how='left')
merged_data.to_csv(os.path.join(proj_path, catalog['output_dir']['dir'], catalog['output_dir']['merged']))

NameError: name 'transactions' is not defined